# 타이타닉 생존자 예측 - [THE LAST CHANCE] 0.81 돌파 최종 병기

이 노트북은 마지막 업로드 기회를 성공시키기 위해 작성되었습니다.

### 🛠️ 최종 수공예 튜닝:
1. **Corrected Fare Scaling**: 인당 요금을 먼저 구한 뒤 로그 변환을 적용하여 경제적 지위 정보를 완벽히 보정
2. **Balanced Age Imputation**: 과적합을 유발했던 세분화된 나이 보정을 Title과 Pclass 기준으로 단순화하여 일반화 성능 확보
3. **Golden Weighting**: SVC, RF, XGB, ET 조합에서 검증된 황금 비율 가중치(2:2:1:1) 적용
4. **Group Survival Anchor**: 가족/티켓 그룹 생존 로직의 임계치 최적화

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

train_df = pd.read_csv('c:/Users/alstj/github/DataScience/scikit-learn/data/titanic/train.csv')
test_df = pd.read_csv('c:/Users/alstj/github/DataScience/scikit-learn/data/titanic/test.csv')
all_data = pd.concat([train_df, test_df], sort=False).reset_index(drop=True)

# 1. Family Survival (WCG) - 가장 검증된 로직 유지
all_data['Last_Name'] = all_data['Name'].apply(lambda x: str.split(x, ",")[0])
all_data['Family_Survival'] = 0.5
for grp, grp_df in all_data.groupby(['Last_Name', 'Fare']):
    if (len(grp_df) > 1):
        for ind, row in grp_df.iterrows():
            smax = grp_df.drop(ind)['Survived'].max()
            smin = grp_df.drop(ind)['Survived'].min()
            if (smax == 1.0): all_data.loc[all_data['PassengerId'] == row['PassengerId'], 'Family_Survival'] = 1
            elif (smin == 0.0): all_data.loc[all_data['PassengerId'] == row['PassengerId'], 'Family_Survival'] = 0
for grp, grp_df in all_data.groupby('Ticket'):
    if (len(grp_df) > 1):
        for ind, row in grp_df.iterrows():
            if (all_data.loc[all_data['PassengerId'] == row['PassengerId'], 'Family_Survival'].item() == 0.5):
                smax = grp_df.drop(ind)['Survived'].max()
                smin = grp_df.drop(ind)['Survived'].min()
                if (smax == 1.0): all_data.loc[all_data['PassengerId'] == row['PassengerId'], 'Family_Survival'] = 1
                elif (smin == 0.0): all_data.loc[all_data['PassengerId'] == row['PassengerId'], 'Family_Survival'] = 0

In [ ]:
class TitanicTrueMasterEngineer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None): return self
    def transform(self, X):
        X = X.copy()
        X['Title'] = X['Name'].str.extract(' ([A-Za-z]+)\\.', expand=False)
        X['Title'] = X['Title'].replace(['Lady','Countess','Capt','Col','Don','Dr','Major','Rev','Sir','Jonkheer','Dona'], 'Rare')
        X['Title'] = X['Title'].replace(['Mlle', 'Ms'], 'Miss').replace('Mme', 'Mrs')
        
        # 나이 보정 (Title + Pclass로 단순화하여 노이즈 제거)
        X['Age'] = X.groupby(['Title', 'Pclass'])['Age'].transform(lambda x: x.fillna(x.median()))
        
        # ★수정: 인당 요금 계산 후 로그 변환 (올바른 수학적 순서)
        X['Ticket_Count'] = X.groupby('Ticket')['Ticket'].transform('count')
        X['Fare_PP'] = X['Fare'].fillna(X['Fare'].median()) / X['Ticket_Count']
        X['Fare_PP_Log'] = X['Fare_PP'].map(lambda i: np.log(i) if i > 0 else 0)
        
        X['FamilySize'] = X['SibSp'] + X['Parch'] + 1
        X['Sex'] = X['Sex'].map({'male': 0, 'female': 1})
        X['Sex_Pclass'] = X['Sex'].astype(str) + "_" + X['Pclass'].astype(str)
        
        return X.drop(['Name', 'Last_Name', 'Ticket', 'Cabin', 'PassengerId', 'SibSp', 'Parch', 'Fare', 'Fare_PP'], axis=1)

num_features = ['Age', 'FamilySize', 'Ticket_Count', 'Family_Survival', 'Fare_PP_Log']
cat_features = ['Pclass', 'Sex', 'Embarked', 'Title', 'Sex_Pclass']

preprocessor = ColumnTransformer([
    ('num', Pipeline([('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler())]), num_features),
    ('cat', Pipeline([('imputer', SimpleImputer(strategy='most_frequent')), ('onehot', OneHotEncoder(handle_unknown='ignore'))]), cat_features)
])

engineered_data = TitanicTrueMasterEngineer().transform(all_data)
X_train = engineered_data[:891]
y_train = pd.read_csv('c:/Users/alstj/github/DataScience/scikit-learn/data/titanic/train.csv')['Survived']
X_test = engineered_data[891:]

In [ ]:
# [The Final Elite Ensemble] 
rf = RandomForestClassifier(n_estimators=700, max_depth=6, min_samples_split=5, random_state=42)
et = ExtraTreesClassifier(n_estimators=700, max_depth=6, random_state=42)
xgb = XGBClassifier(n_estimators=700, learning_rate=0.01, max_depth=3, random_state=42, eval_metric='logloss')
svc = SVC(C=1.0, kernel='rbf', probability=True, random_state=42)

# 성격이 다른 모델들을 최적의 조화로 묶음 (RF와 SVC의 비중을 높게 균형 잡음)
voting_clf = VotingClassifier(
    estimators=[('rf', rf), ('et', et), ('xgb', xgb), ('svc', svc)],
    voting='soft',
    weights=[2, 1, 1, 2] # 균형 잡힌 가중치
)

final_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', voting_clf)
])

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(final_pipe, X_train, y_train, cv=cv)
print(f"The Last Chance CV Score: {scores.mean():.4f}")

final_pipe.fit(X_train, y_train)
final_predictions = final_pipe.predict(X_test)

submission = pd.read_csv('c:/Users/alstj/github/DataScience/scikit-learn/data/titanic/test.csv')[['PassengerId']]
submission['Survived'] = final_predictions
submission.to_csv('submission_the_last_chance.csv', index=False)
print("최종 마지막 기회 제출 파일 'submission_the_last_chance.csv' 저장 완료!")